In [2]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from scipy import optimize
dp = '/home/ebert/jupyter-notebooks/FNode-data'
exclude = [2]

def gaussian(x, a, x0, s, b):
    return b + a*np.exp(-0.5*((x-x0)/s)**2)

In [3]:
exp_name = ''
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
cs_cuts = [[400],[3]]
rb_cuts = [[300],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
retention = q.apply_thresholds(exclude_rois=exclude)
cuts = q.get_thresholds()
roi_map = ['cs', 'rb']
fmt_map = ['bo', 'rx']
entries = np.product(retention['retention'].shape[:-1])
tbl_str = "ivar: {0:.4f}\tcloading: {1:.3f}\trloading: {2:.3f}\t\tcretention: {3:.3f} +- {4:.3f}\trretention: {5:.3f} +- {6:.3f}"
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['loading'].reshape((entries,-1))[i][0], retention['loading'].reshape((entries,-1))[i][1],
        retention['retention'].reshape((entries,-1))[i][0], retention['error'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i][1], retention['error'].reshape((entries,-1))[i][1]
    ))
q.save_experiment_data()
xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']

# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate(['retention', 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals=='retention':
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention['error'][e,:,r],
                        fmt=fmt_map[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.6, retention['ivar'][e, np.argmin(retention[yvals][e, :, r]), 0], 0.4, .0]
                try:
                    popt, pcov = optimize.curve_fit(
                        gaussian,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
                        sigma=retention['error'][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, gaussian(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\nx0: {:.3f}\ns: {:.3f}\nb: {:.3f}"
                    start = np.min(retention['ivar'][e, :, 0])
                    stop = np.max(retention['ivar'][e, :, 0])
                    dx = (stop-start)/10
                    ax[i].text(start+dx+r*5*dx, 0.5, fit_data.format(roi_map[r],*popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                
        
    ax[i].set_ylim(0., 1)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'loading_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_05_12/2018_05_12_14_29_45_ssro-ix/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'Ix_read']
ivar: 0.1500	cloading: 0.185	rloading: 0.210		cretention: 0.889 +- 0.024	rretention: 0.664 +- 0.024
ivar: 0.1800	cloading: 0.306	rloading: 0.181		cretention: 0.897 +- 0.024	rretention: 0.566 +- 0.024
ivar: 0.2100	cloading: 0.249	rloading: 0.175		cretention: 0.913 +- 0.025	rretention: 0.625 +- 0.025
ivar: 0.2400	cloading: 0.263	rloading: 0.188		cretention: 0.829 +- 0.024	rretention: 0.669 +- 0.024
ivar: 0.2700	cloading: 0.156	rloading: 0.211		cretention: 0.926 +- 0.024	rretention: 0.668 +- 0.024
ivar: 0.3000	cloading: 0.162	rloading: 0.224		cretention: 0.836 +- 0.024	rretention: 0.681 +- 0.024
ivar: 0.3300	cloading: 0.168	rloading: 0.246		cretention: 0.816 +- 0.023	rretention: 0.737 +- 0.023
ivar: 0.3600	cloading: 0.239	rloading: 0.198		cretention: 0.865 +- 0.024	rretention: 0.674 +- 0.024
ivar: 0.3900	cloading: 0.144	rloading: 0.182		creten

/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:118: RuntimeWarning: overflow encountered in double_scalars
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))
/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:118: RuntimeWarning: invalid value encountered in sqrt
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))
/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:275: RuntimeWarning: divide by zero encountered in divide
  e['iterations'][i]['retention'] = retention/loaded
/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:275: RuntimeWarning: invalid value encountered in divide
  e['iterations'][i]['retention'] = retention/loaded


FigureCanvasNbAgg()

[ 0.12100629  0.17134076  0.14960064  0.77290944]
[ 0.09410237  0.32041971 -0.05082406  0.61669192]


In [3]:
# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate(['retention', 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals=='retention':
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention['error'][e,:,r],
                        fmt=fmt_map[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.6, retention['ivar'][e, np.argmin(retention[yvals][e, :, r]), 0], 0.2, .6]
                try:
                    popt, pcov = optimize.curve_fit(
                        gaussian,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
#                         sigma=retention['error'][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, gaussian(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\nx0: {:.3f}\ns: {:.3f}\nb: {:.3f}"
                    start = np.min(retention['ivar'][e, :, 0])
                    stop = np.max(retention['ivar'][e, :, 0])
                    dx = (stop-start)/10
                    ax[i].text(start+dx+r*6*dx, 0.1, fit_data.format(roi_map[r],*popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                
        
    ax[i].set_ylim(0., 1.05)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'loading_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

[ -3.01166383e+02  -3.14593595e-02  -5.04097583e+00   3.01966334e+02]
[-0.05794977 -0.0061385  -0.04389579  0.59028975]
